## 处理数据

In [1]:
import pandas as pd

In [2]:
content = pd.read_csv('qa_processed', encoding='utf-8')

In [3]:
content.head()

,qid,question,answer,q_processed
0,1,补发网银盾,您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。,补发 网银 盾
1,2,代发工资,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。,代发 工资
2,3,对帐单查询打印,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。,对 帐单 查询 打印
3,4,理财产品取号,请说出您需要办理的业务,理财 产品 取号
4,5,密码修改,请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。,密码 修改


In [10]:
# 数据集的index（索引）还是按照正常的排序。no need
content = content.sample(frac=1).reset_index(drop=True)

In [11]:
# from sklearn.utils import shuffle
# shuffle(content)

In [37]:
import re


def split_to_sentence(content):
    "分句"
    pattern = re.compile(".*?[。.？?！!]")  # 非贪婪模式匹配文字内容
    sents = re.findall(pattern, content)
    return sents


def basic_clean(content):
    content = content.lower()
    
    # 去除重复字符
    content = re.sub('(\n)+','\n',content)
    content = re.sub('(\r)+','',content)
    content = re.sub('(\t)+','，',content)

    content = re.sub('[？]+?[！]+?','',content)
    content = re.sub('[？]+?[。]+?','',content)
    content = re.sub('(～)+','～',content)
    content = re.sub('(，)+','，',content)
    content = re.sub('(。)+','。',content)
    content = re.sub('(～。)+','。',content)
    content = re.sub('(？)+','？',content)
    content = re.sub('(！)+','！',content)
    content = re.sub('(；)+','；',content)
    content = re.sub('( )+',' ',content).strip()

    return content

In [38]:
# 格式：一行一句话；每个字符空格隔开

with open('corpus_task', 'w', encoding='utf-8') as f:
    for idx, row in content.iterrows():
        line = ''
        pre = '@'
        
        q = basic_clean(str(row.question))
        a = basic_clean(str(row.answer))
        
        q_list = split_to_sentence(q)
        q = q_list if len(q_list) > 0 else [q]
        
        a_list = split_to_sentence(a)
        a = a_list if len(a_list) > 0 else [a]        
        
        for qi in q:
            for s in str(qi).lower():
                if ord('A') <= ord(pre) <= ord('z') and  ord('A') <= ord(s) <= ord('z'):
                    line += s
                else:
                    line += ' ' + s
                pre = s
            f.write(line + '\n')
        for ai in a:
            for s in str(ai).lower():
                if ord('A') <= ord(pre) <= ord('z') and  ord('A') <= ord(s) <= ord('z'):
                    line += s
                else:
                    line += ' ' + s
                pre = s
            f.write(line + '\n')
        
        if idx % 5000 == 0:
            print(line)

 我 们 去 抢 劫 吧 哎 ， 我 最 腻 味 这 种 没 有 技 术 含 量 的 活 儿 ～
 不 理 我 就 算 了 呵 呵 ， 偶 可 不 可 以 不 回 答 这 个 问 题 啊 ， , 不 如 我 给 您 做 心 理 测 试 吧 ？
 心 灰 意 冷 暖 暖 心
 你 几 点 上 班 小 龙 人 二 十 四 小 时 为 您 服 务 。
 这 就 是 天 意 天 意 难 违 啊
 汽 车 配 置 中 的 at 是 什 么 自 动 挡 汽 车
 能 拍 张 照 片 吗 小 龙 人 好 高 兴 ， 这 是 我 的 荣 幸
